> Projeto Desenvolve <br>
Programação Intermediária com Python <br>
Profa. Camila Laranjeira (mila@projetodesenvolve.com.br) <br>

# 3.14 - ORM

## Exercícios

#### Q1. Conhecendo os dados
Baixe o seguinte csv onde iremos trabalhar. Ele contém informações sobre salários de profissionais de dados de uma empresa hipotética entre 2009 e 2016
* https://github.com/camilalaranjeira/python-intermediario/blob/main/salaries.csv

Suas colunas, descritas na [página do Kaggle que contém o dataset](https://www.kaggle.com/datasets/krishujeniya/salary-prediction-of-data-professions?resource=download), são:
* FIRST NAME: Primeiro nome do profissional de dados (String)
* LAST NAME: Sobrenome do profissional de dados (String)
* SEX: Gênero do profissional de dados (String: 'F' para Feminino, 'M' para Masculino)
* DOJ (Date of Joining): A data em que o profissional de dados ingressou na empresa (Data no formato MM/DD/AAAA)
* CURRENT DATE: A data atual ou a data de referência dos dados (Data no formato MM/DD/AAAA)
* DESIGNATION: O cargo ou designação do profissional de dados (String: ex., Analista, Analista Sênior, Gerente)
* AGE: Idade do profissional de dados (Integer)
* SALARY: Salário anual do profissional de dados (Float)
* UNIT: Unidade de negócios ou departamento em que o profissional de dados trabalha (String: ex., TI, Finanças, Marketing)
* LEAVES USED: Número de licenças utilizadas pelo profissional de dados (Integer)
* LEAVES REMAINING: Número de licenças restantes para o profissional de dados (Integer)
* RATINGS: Avaliações de desempenho do profissional de dados (Float)
* PAST EXP: Experiência de trabalho anterior em anos antes de ingressar na empresa atual (Float)

Na célula a seguir, **carregue os dados do CSV e dê uma olhada neles antes de seguir**.

In [146]:
import pandas as pd

df=pd.read_csv('salaries.csv')
print(df)


     FIRST NAME   LAST NAME SEX         DOJ CURRENT DATE     DESIGNATION  \
0        TOMASA       ARMEN   F   5-18-2014   01-07-2016         Analyst   
1         ANNIE         NaN   F         NaN   01-07-2016       Associate   
2         OLIVE        ANCY   F   7-28-2014   01-07-2016         Analyst   
3        CHERRY     AQUILAR   F  04-03-2013   01-07-2016         Analyst   
4          LEON  ABOULAHOUD   M  11-20-2014   01-07-2016         Analyst   
...         ...         ...  ..         ...          ...             ...   
2634  KATHERINE      ALSDON   F   6-28-2011   01-07-2016  Senior Manager   
2635     LOUISE     ALTARAS   F   1-14-2014   01-07-2016         Analyst   
2636      RENEE      ALVINO   F   1-23-2014   01-07-2016         Analyst   
2637       TERI   ANASTASIO   F   3-17-2014   01-07-2016         Analyst   
2638    GREGORY      ABARCA   M   9-18-2014   01-07-2016         Analyst   

       AGE  SALARY        UNIT  LEAVES USED  LEAVES REMAINING  RATINGS  \
0     21.0   

#### Q2. Modelando os dados
Você deve **criar um ORM com SQLAlchemy capaz de comportar os dados dessa base**.

* Crie um campo de chave primária `ID`, que deve ser incrementado automaticamente
* Os campos SEX, DESIGNATION e UNIT devem ser definidos como classes `Enum` com os possíveis valores (consulte os valores únicos dessas colunas)
* Para os outros campos, consulte os tipos de dados informados na descrição acima

In [147]:
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column
from sqlalchemy import Enum as SQLEnum
from datetime import date
from enum import Enum

class Genero(Enum):
    F = "Feminino"
    M = "Masculino"

class Designacao(Enum):
    ANALYST = "Analyst"
    SENIOR_ANALYST = "Senior Analyst"
    ASSOCIATE = 'Associate'
    MANAGER = "Manager"
    DIRECTOR = "Director"
    SENIOR_MANAGER = "Senior Manager"

class Setor(Enum):
    IT = "IT"
    FINANCE = "Finance"
    MARKETING = "Marketing"
    WEB = "Web"
    MANAGEMENT = "Management"
    OPERATIONS = "Operations"

class Base(DeclarativeBase):
    pass

class DataFuncionario(Base):
    __tablename__ = 'funcionario'
    
    ID: Mapped[int]= mapped_column(primary_key=True)
    first_name: Mapped[str] = mapped_column(nullable=False, name='FIRST NAME')
    last_name: Mapped[str] = mapped_column(nullable=False)
    sex: Mapped[Genero] =mapped_column(SQLEnum(Genero), nullable=False)
    doj: Mapped[date] = mapped_column (nullable=False)
    current_date: Mapped[date] = mapped_column (nullable=False)
    designation: Mapped[Designacao] =mapped_column(SQLEnum(Designacao), nullable=False)
    age: Mapped[int] = mapped_column(nullable=False)
    salary: Mapped[float] = mapped_column(nullable=False)
    unit: Mapped[Setor] =mapped_column(SQLEnum(Setor), nullable=False)
    leaves_used: Mapped[int] = mapped_column(nullable=False)
    leaves_remaining: Mapped[int] = mapped_column(nullable=False)
    ratings: Mapped[float] = mapped_column(nullable=False)
    past_exp: Mapped[float] = mapped_column(nullable=False)




#### Q3. Estabelecendo uma conexão

Usando o método `create_engine` do SQLAlchemy, crie uma conexão com um novo banco de dados SQLite chamado `salarios`.

In [148]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///salarios2.db')


#### Q4. Criando as tabelas
Crie as tabelas da questão Q2 no banco `salarios`.

In [149]:
Base.metadata.create_all(engine)

#### Q5. Populando

Usando o método `to_sql` da biblioteca Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)), popule o banco `salarios` com os dados do csv que você carregou na questão Q1.
* Lembre-se de definir o parâmetro `if_exists='append'` para que as tabelas não sejam dropadas e recriadas.

In [150]:
df.columns = [col.upper() for col in df.columns]
df.to_sql('funcionario', con=engine, if_exists='append', index=False)

2639

#### Q6. Consultas SQL vs ORM

Agrupe os dados por DESIGNATION e selecione o mínimo, máximo e a média dos salários (SALARY) divididos por 12. Já que o atributo SALARY é anual, dividir por 12 nos mostrará os valores mensais.

Assumindo que a variável que armazena a sua conexão se chama `engine`, você deve realizar a query acima de três formas:
* Executando a query SQL através de uma instância de conexão retornada pelo método `engine.connect()`
* Executando a query SQL com o método `read_sql_query` do Pandas (veja [a documentação](https://pandas.pydata.org/docs/reference/api/pandas.read_sql_query.html)). Você usará mesma instância `engine.connect()` como um dos parâmetros.
* Executando uma query criada com o módulo `select` do SQLAlchemy. Sua execução deve ser feita através de um objeto `Session` do módulo `orm` do SQLAlchemy (`Session(engine)`).


In [151]:
from sqlalchemy import text
with engine.connect() as connection:
    query = """
    SELECT 
        designation,
        MIN(salary / 12) AS minimo_mensal,
        MAX(salary / 12) AS maximo_mensal,
        AVG(salary / 12) AS media_mensal
    FROM funcionario
    GROUP BY designation
    """
    

    result = connection.execute(text(query))
    
    for row in result:
        print(f"Designação: {row[0]}, Mínimo: {row[1]:.2f}, Máximo: {row[2]:.2f}, Média: {row[3]:.2f}")


Designação: Analyst, Mínimo: 3333.00, Máximo: 4165.00, Média: 3751.22
Designação: Associate, Mínimo: 5846.00, Máximo: 8300.00, Média: 7266.47
Designação: Director, Mínimo: 17832.00, Máximo: 32342.00, Média: 23913.94
Designação: Manager, Mínimo: 8343.00, Máximo: 12407.00, Média: 10522.25
Designação: Senior Analyst, Mínimo: 4170.00, Máximo: 5830.00, Média: 4991.33
Designação: Senior Manager, Mínimo: 12614.00, Máximo: 16631.00, Média: 14888.23


In [152]:
with engine.connect() as connection:
    df_result = pd.read_sql_query(sql=text(query), con=connection)
print(df_result)

      DESIGNATION  minimo_mensal  maximo_mensal  media_mensal
0         Analyst           3333           4165   3751.217034
1       Associate           5846           8300   7266.465409
2        Director          17832          32342  23913.937500
3         Manager           8343          12407  10522.246914
4  Senior Analyst           4170           5830   4991.325843
5  Senior Manager          12614          16631  14888.225806


In [153]:
from sqlalchemy.orm import Session
from sqlalchemy import func, select

with Session(engine) as session:
    query = (
        select(
            func.upper(DataFuncionario.designation).label("DESIGNATION"),
            func.min(DataFuncionario.salary / 12).label("mininmo_mensal"),
            func.max(DataFuncionario.salary / 12).label("maximo_mensal"),
            func.avg(DataFuncionario.salary / 12).label("media_mensal"),
        )
        .group_by(DataFuncionario.designation)
    )
    
    result = session.execute(query)

    for row in result:
        print(f"Designação: {row[0]}, Mínimo: {row[1]:.2f}, Máximo: {row[2]:.2f}, Média: {row[3]:.2f} ")


Designação: ANALYST, Mínimo: 3333.42, Máximo: 4165.00, Média: 3751.68 
Designação: ASSOCIATE, Mínimo: 5846.17, Máximo: 8300.25, Média: 7266.92 
Designação: DIRECTOR, Mínimo: 17832.25, Máximo: 32342.67, Média: 23914.27 
Designação: MANAGER, Mínimo: 8343.67, Máximo: 12407.50, Média: 10522.72 
Designação: SENIOR ANALYST, Mínimo: 4170.33, Máximo: 5830.50, Média: 4991.78 
Designação: SENIOR MANAGER, Mínimo: 12614.42, Máximo: 16631.42, Média: 14888.69 
